Prvi del najinega projekta

In [24]:
import random
import sage.graphs.graph_plot
from sage.graphs.base.boost_graph import *
import numpy

Funkcija, ki nam vrne c-je kot seznam seznamov

In [25]:
def seznam_cen(st_opazanj, st_pov_grafa, min_cena, max_cena):
    sur_podatki = []
    for i in range(0, st_opazanj):
        mer_povezav = []
        for j in range(0, st_pov_grafa):
            mer_povezav.append(random.uniform(min_cena, max_cena))
        sur_podatki.append(mer_povezav)
    return(sur_podatki)

#print(seznam_cen(5, 10, 0, 10))

Funkcija, ki nam zmnoži vektor z transponiranim vektorjem

In [26]:
def mnoz(vektor1, vektor2):
    matrika = []
    for i in range(0,len(vektor1)):
        vrstica =[]
        for j in range(0, len(vektor2)):
            vrstica.append(vektor1[i] * vektor2[j])
        matrika.append(vrstica)
    return(matrika)

Kovariancna matrika

In [28]:
def kov_matrika(vse_utezi):
    avr_pov = [i * 1/len(vse_utezi) for i in [sum(x) for x in zip(*vse_utezi)]]
    zac_mat = numpy.matrix(mnoz(numpy.subtract(vse_utezi[0],avr_pov),numpy.subtract(vse_utezi[0],avr_pov)))
    for i in range(1, len(vse_utezi)):
        zac_mat += numpy.matrix(mnoz(numpy.subtract(vse_utezi[i],avr_pov),numpy.subtract(vse_utezi[i],avr_pov)))
    kon_mat = 1/len(vse_utezi)*zac_mat
    return(kon_mat)

#diagonala:
#numpy.diag(testna_kov_mat)


Generiramo sosednosti (vozlišča za graf)

In [29]:
def nakljucno_vozlisce(st_vozlisc1):
    start = random.randint(0, st_vozlisc1)
    end = random.randint(0,st_vozlisc1)
    if start == end:
        return nakljucno_vozlisce(st_vozlisc1)
    else:
        return[start, end]

#nakljucno_vozlisce(ggrafi[8])

def vsa_vozlisca(koliko, st_pov1):
    vozlisca = []
    for i in range(0, st_pov1):
        x = nakljucno_vozlisce(koliko)
        while x in vozlisca:
            x = nakljucno_vozlisce(koliko)
        vozlisca.append(x)
    return vozlisca

Generiramo naključen graf:

In [30]:
def graf(st_opazanj, st_vozlisc, st_pov_grafa, min_cena, max_cena):
    utezi2 = seznam_cen(st_opazanj, st_pov_grafa, min_cena, max_cena)
    #graf_seznam = graphs.RandomGNM(st_vozlisc, st_pov_grafa).edges(labels=False)
    graf_seznam =  vsa_vozlisca(st_vozlisc, st_pov_grafa)
    nov = [] #najprej spremeniva v list
    for i in range(0,len(graf_seznam)):
        nov.append(list(graf_seznam[i]))
    konec = [] #dodava še ceno, v obliki ki jo sprjema digraph
    seznam_gra = []
    for j in range(0,len(utezi2)):
        zadnji = []
        for i in range(0,len(nov)):
            vmesni = []
            vmesni.append(nov[i][0])
            vmesni.append(nov[i][1])
            vmesni.append(utezi2[j][i])
            zadnji.append(tuple(vmesni))
        konec.append(zadnji)
        gr = DiGraph(zadnji)
        seznam_gra.append(gr)
    #seznam_gra[0].show(edge_labels = True)
    return(seznam_gra, utezi2)

#graf(5, 15, 150, 1, 5)[0][0].show(edge_labels = True)

Vektorji poti x

In [31]:
def vektorcki_x(graf, zacetno, koncno):
    vse_poti = graf.all_paths(zacetno, koncno, use_multiedges=True, report_edges=False, labels=False)
    vse_povezave = graf.edges()
    par = []
    for i in range(0,len(vse_povezave)):
        par.append(vse_povezave[i][0:2])
    seznam_vektorjev = []
    for i in range(0, len(vse_poti)):
        pot = (vse_poti[i])
        prvi_oklepaj1 = list(zip(pot, pot[1:] + pot[:1]))
        prvi_oklepaj = prvi_oklepaj1[:-1]
        x=[]
        for i in range(0, len(par)):
            if par[i] in prvi_oklepaj:
                x.append(1)
            else:
                x.append(0)
        seznam_vektorjev.append(x)
    return(seznam_vektorjev)

#x = ((graf(5, 5, 10, 1, 5))[0][0])
#x.show()
#vektorcki_x(x, 2, 4)

Izračunamo ceno določene poti

In [32]:
def povprecje(cene1):
    seznam_cen = list(cene1.values())
    povprecne_cene = []
    for i in range(0, len(seznam_cen)):
        povprecne_cene.append(numpy.average(seznam_cen[i]))
    return(povprecne_cene)


def vektor_skalarni_produkt(cene1, start, end, graf):
    cene_poti = {}
    for i in range(0,len(cene1)):
        x = vektorcki_x(graf[0], start, end)
        vmesni = []
        for j in range(0,len(x)):
            vmesni.append(numpy.dot(cene[i], x[j]))
        cene_poti[i] = vmesni
    return




Celotna priprava vseh reči, ki jih potrebujemo neposredno za naivni algoritem

In [116]:
#prvo narediva vektorja xr in xl, za začetek najinega algoritma
#graf(st_opazanj, st_vozlisc, st_pov_grafa, min_cena, max_cena)

[seznam_gra, utezi] = graf(5, 4, 15, 0, 10)
#velikost d in avr_pov ga ne moti
d = numpy.diag(kov_matrika(utezi))
avr_pov = [i * 1/len(utezi) for i in [sum(x) for x in zip(*utezi)]]
E = 0.3

x = vektorcki_x(seznam_gra[0], 1, 4)

#VIDALIJEV PREDLOG ZA ARGLEXMIN
def arglexmin(seznam_x, vektorji):
    xmin = min(([numpy.dot(numpy.array(v), numpy.array(x)) for v in vektorji], x) for x in seznam_x)
    return (xmin, xmin[0][0]+xmin[0][1])


#[najmanjsi_xl, cenal] = arglexmin(x, [avr_pov, d])
[najmanjsi_xl, cenal] = arglexmin(x, [numpy.dot((1-E), avr_pov), numpy.dot(E, d)])
najmanjsi_xl_pravi = najmanjsi_xl[1]
print(najmanjsi_xl_pravi)


#[najmanjsi_xr, cenar] = arglexmin(x, [d, avr_pov])
[najmanjsi_xr, cenar] = arglexmin(x, [numpy.dot((1-E), d),numpy.dot(E, avr_pov)])
najmanjsi_xr_pravi = najmanjsi_xr[1]
print(najmanjsi_xr_pravi)



[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [105]:
#Prvotna funkcija, pred popravki asistent(je ne uporabljamo v nadaljevanju)

def najmanjsi_x(seznam_x, vektor):
    najmanjsi_x = seznam_x[0]
    najmanjsi_skalarni_produkt = numpy.dot(vektor, numpy.array(seznam_x[0]))
    for i in range(1,len(seznam_x)):
        vrednost = numpy.dot(vektor, numpy.array(seznam_x[i]))
        if vrednost < najmanjsi_skalarni_produkt:
            najmanjsi_skalarni_produkt = vrednost
            najmanjsi_x = seznam_x[i]
    return(najmanjsi_x)

najmanjsi_xl2 = najmanjsi_x(x, vektor_xl)
najmanjsi_xr2 = najmanjsi_x(x, vektor_xr)
print(najmanjsi_xl2)
print(najmanjsi_xr2)


#vektor_xl =  numpy.add(numpy.dot((1-E), numpy.array(avr_pov)), numpy.dot(E,numpy.array(d)))
#print(vektor_xl)
#xl= numpy.dot(vektor_xl, numpy.array(x[0])) #prvi vektor
#print(xl)

#vektor_xr = numpy.add(numpy.dot(E, numpy.array(avr_pov)), numpy.dot((1-E),numpy.array(d)))
#xr= numpy.dot((vektor_xr), numpy.array(x[0]))
#print(xr)


[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [86]:
import sys
sys.setrecursionlimit(10000)
print(sys.getrecursionlimit())


10000


Naivni algoritem

In [131]:
uer3 = []
def razisci3(x0, x1, pov, diag, poti):
    if x0 == x1:
        print("x0 in x1 sta enaka.")
    else:
        alfa_m = numpy.dot(diag,numpy.subtract(x1,x0)) / numpy.dot(numpy.subtract(pov,diag),numpy.subtract(x0,x1))
        vek_zv = numpy.add(numpy.dot(alfa_m, numpy.array(pov)), numpy.dot((1-alfa_m),numpy.array(diag)))
        v_zv_alfa = numpy.dot(alfa_m, numpy.array(pov))
        v_zv_ena_minus_alfa = numpy.dot((1-alfa_m),numpy.array(diag))
        x_zv = arglexmin(poti, [v_zv_alfa, v_zv_ena_minus_alfa])[0][1]
        #print(x_zv)
        #print(numpy.dot(vek_zv, x_zv))
        #print(numpy.dot(vek_zv, x0))
        if numpy.dot(vek_zv, x_zv) < numpy.dot(vek_zv, x0):
            print("Najdeno je bilo izboljšanje.")
            uer3.append(x_zv)
            razisci(x0, x_zv, pov, diag, poti)
            razisci(x_zv, x1, pov, diag, poti)
        else:
            print("Izboljšava ni najdena.")
            uer3.append(x0)
    return(print(uer3))


uc_res = razisci3(najmanjsi_xl_pravi, najmanjsi_xr_pravi, avr_pov, d, x)


Izboljšava ni najdena.
[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]
